<h1>Proyecto Tecnologías de Búsqueda en la Web</h1>
<h3>Integrantes</h3>
<ul><li>Sebastián Aranda</li><li>Felipe Santander</li></ul>

<h2>Librerías</h2>

In [1]:
import gensim
from gensim import corpora
from gensim.corpora import Dictionary, bleicorpus
from gensim.models import ldamodel
from gensim.models import Phrases

from collections import defaultdict

import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import matplotlib.pyplot as plt
import numpy as np

from random import randint
from __future__ import division

import pyLDAvis
import re
import os
import json
import operator

<h2>Creación del Corpus</h2>

<h6>Rutas a Datasets</h6>

In [5]:
negative_dataset_path = "Corpus_builder_and_pre/Dataset/Bullying"
positive_dataset_path = "Corpus_builder_and_pre/Dataset/NoBullying"
#balance_dataset_path = "Corpus_builder_and_pre/Neutros&Positivos/"

<h6>Construcción del Corpus</h6>

In [7]:
puncts = ".,:;?!()[]{}~+-\"\'#$%&\/*^|"
digits = "0123456789"
#Create bad words list
bad_words = []
with open('badwords_v2.txt','r') as bad_words_file:
    for word in bad_words_file:
        word = word.split('-')[0].strip()
        word = word.replace('\n','').decode('unicode_escape').encode('ascii','ignore')
        if word != '':
            bad_words.append(word)

#Create the corpus
corpus = list()
for name in os.listdir(positive_dataset_path):
    if name.endswith('.json'):
        with open(positive_dataset_path+'/'+name) as f:
            op_json = json.loads(f.read())
            try:
                for child in op_json[1]['data']['children']:    
                    #Extract the comment
                    comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                    #Delete links
                    comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    
                    for sym in puncts:
                        comment_text = comment_text.replace(sym," ")
                    for num in digits:
                        comment_text = comment_text.replace(num," ")
                    
                    tokens_comment = [word for word in comment_text.lower().split()]        
                    corpus.append(tokens_comment)
                    
                    try:
                        for child in child['data']['replies']['data']['children']:
                            comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                            comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    
                            for sym in puncts:
                                comment_text = comment_text.replace(sym," ")
                            for num in digits:
                                comment_text = comment_text.replace(num," ")
                        
                            tokens_comment = [word for word in comment_text.lower().split()]
                            corpus.append(tokens_comment)
                            
                    except Exception as e:
                        pass
                    
            except Exception as e:
                pass

positive_dataset_size = len(corpus)
print(len(corpus))

for name in os.listdir(negative_dataset_path):
    if name.endswith('.json'):
        with open(negative_dataset_path+'/'+name) as f:
            op_json = json.loads(f.read())
            try:
                for child in op_json[1]['data']['children']:
                    
                    if (len(corpus)>2*positive_dataset_size):
                        break
                    
                    comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                    comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    
                    for sym in puncts:
                        comment_text = comment_text.replace(sym," ")
                    for num in digits:
                        comment_text = comment_text.replace(num," ")
                    
                    tokens_comment = [word for word in comment_text.lower().split()]    
                    corpus.append(tokens_comment)
                    
                    try:
                        for child in child['data']['replies']['data']['children']:
                            if (len(corpus)>2*positive_dataset_size):
                                break
                            
                            comment_text = child['data']['body'].encode('ascii', 'ignore').replace('\n', ' ')
                            comment_text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', comment_text)
                    
                            for sym in puncts:
                                comment_text = comment_text.replace(sym," ")
                            for num in digits:
                                comment_text = comment_text.replace(num," ")
                        
                            tokens_comment = [word for word in comment_text.lower().split()]    
                            corpus.append(tokens_comment)             
                    except Exception as e:
                        #print(e)
                        pass
                    
            except Exception as e:
                #print(e)
                pass

print(len(corpus))

48859
97719


<h6>Procesamiento del Corpus</h6>

In [8]:
#Remove lone letters
corpus = [[word for word in doc if len(word)>1] for doc in corpus]

#Remove stopwords
corpus = [[word for word in doc if word not in stopwords.words('english')] for doc in corpus]

# Lemmatize all words in documents.
lemmatizer = WordNetLemmatizer()
corpus = [[lemmatizer.lemmatize(word) for word in doc] for doc in corpus]

#Get Trigrams
#bigram = Phrases(corpus)
#trigram = Phrases(bigram[corpus])
#for idx in range(len(corpus)):
#    for token in trigram[corpus[idx]]:
#            if '_' in token:
#                corpus[idx].append(token)

<h6>Transformación del Corpus a espacio de vectores</h6>

In [9]:
#Create dictionary
dictionary = corpora.Dictionary(corpus)
dictionary.filter_extremes()

#Convert documents to vectors
corpus = [dictionary.doc2bow(doc) for doc in corpus]

#Save corpus and dictionary
corpora.BleiCorpus.serialize('tmp/cyberbullying_corpus.lda-c', corpus)
dictionary.save('tmp/cyberbullying_dictionary.dict')

<h1>Características para Clasificación</h1>
<ul>
<li>Porcentaje de pertenencia al tópico generado de LDA en Roastme Dataset</li>
<li>Densidad de Badwords</li>
<li>TODO: Polaridad</li>
</ul>

In [28]:
#Load the dictionary and corpus
if (os.path.exists('tmp/cyberbullying_dictionary.dict') and os.path.exists('tmp/cyberbullying_corpus.lda-c')):
    print('Creating dictionary...')
    %time dictionary = corpora.Dictionary.load('tmp/cyberbullying_dictionary.dict')
    print('Creating corpus...')
    %time corpus = corpora.BleiCorpus('tmp/cyberbullying_corpus.lda-c')
    print("Corpus Size: "+str(len(corpus)))
else:
    print("Create the dictionary and corpus first")

Creating dictionary...
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 674 µs
Creating corpus...
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.66 ms
Corpus Size: 601


<h3>LDA Features</h3>

In [13]:
#Initialize a model
#print('Creating Tfidf model...')
#tfidf = gensim.models.TfidfModel(corpus)
#corpus_tfidf = tfidf[corpus]
#lda_model_tfidf = ldamodel.LdaModel(corpus_tfidf, num_topics=2, id2word=dictionary)

#LDA-2
%time lda_model = ldamodel.LdaModel(corpus, num_topics=2, id2word=dictionary)
lda_model.save('tmp/cyberbullying_ldaModel.lda')
lda_model.print_topics(2,40)

CPU times: user 2min 41s, sys: 364 ms, total: 2min 41s
Wall time: 2min 41s


[(0,
  u'0.048*"like" + 0.042*"look" + 0.011*"one" + 0.010*"get" + 0.009*"face" + 0.009*"guy" + 0.007*"would" + 0.006*"girl" + 0.006*"make" + 0.006*"know" + 0.005*"kid" + 0.005*"fuck" + 0.005*"think" + 0.005*"even" + 0.005*"say" + 0.005*"roast" + 0.005*"time" + 0.005*"got" + 0.005*"school" + 0.004*"tell" + 0.004*"go" + 0.004*"people" + 0.004*"going" + 0.004*"year" + 0.004*"head" + 0.004*"thing" + 0.004*"right" + 0.004*"good" + 0.004*"see" + 0.004*"life" + 0.004*"friend" + 0.004*"could" + 0.004*"back" + 0.004*"bet" + 0.004*"already" + 0.004*"still" + 0.003*"left" + 0.003*"shit" + 0.003*"want" + 0.003*"probably"'),
 (1,
  u'0.013*"hair" + 0.009*"post" + 0.008*"look" + 0.007*"face" + 0.007*"please" + 0.005*"like" + 0.005*"oh" + 0.005*"eyebrow" + 0.004*"love" + 0.004*"use" + 0.004*"subreddit" + 0.004*"forehead" + 0.004*"comment" + 0.004*"eye" + 0.004*"teeth" + 0.004*"report" + 0.004*"black" + 0.004*"god" + 0.004*"deleted" + 0.003*"see" + 0.003*"thought" + 0.003*"guy" + 0.003*"hate" + 0.003

In [34]:
#LDA-3
%time lda_model = ldamodel.LdaModel(corpus, num_topics=3, id2word=dictionary)
lda_model.save('tmp/cyberbullying_ldaModel.lda')
lda_model.print_topics(3,20)

CPU times: user 3.44 s, sys: 12 ms, total: 3.45 s
Wall time: 3.46 s


[(0,
  u'0.028*"hair" + 0.028*"like" + 0.024*"really" + 0.021*"get" + 0.020*"looking" + 0.018*"change" + 0.017*"girl" + 0.017*"good" + 0.016*"look" + 0.016*"beard" + 0.016*"also" + 0.016*"right" + 0.015*"always" + 0.014*"think" + 0.013*"want" + 0.013*"wow" + 0.013*"better" + 0.013*"school" + 0.013*"tell" + 0.012*"got"'),
 (1,
  u'0.056*"like" + 0.030*"look" + 0.028*"get" + 0.020*"still" + 0.018*"face" + 0.018*"picture" + 0.017*"thing" + 0.014*"ever" + 0.013*"year" + 0.013*"even" + 0.013*"much" + 0.013*"damn" + 0.013*"mean" + 0.013*"holy" + 0.012*"see" + 0.012*"would" + 0.012*"transformation" + 0.012*"make" + 0.012*"time" + 0.011*"say"'),
 (2,
  u'0.101*"look" + 0.084*"like" + 0.024*"ugly" + 0.018*"nice" + 0.018*"one" + 0.017*"would" + 0.015*"could" + 0.015*"think" + 0.015*"year" + 0.014*"amish" + 0.014*"life" + 0.013*"go" + 0.013*"way" + 0.012*"fuck" + 0.011*"guy" + 0.011*"know" + 0.011*"great" + 0.011*"face" + 0.010*"first" + 0.010*"self"')]

In [11]:
#LDA-5
%time lda_model = ldamodel.LdaModel(corpus, num_topics=5, id2word=dictionary)
lda_model.save('tmp/cyberbullying_ldaModel.lda')
lda_model.print_topics(5,20)

CPU times: user 3min 22s, sys: 384 ms, total: 3min 22s
Wall time: 3min 22s


[(0,
  u'0.018*"post" + 0.015*"please" + 0.015*"gay" + 0.015*"dad" + 0.009*"chin" + 0.009*"subreddit" + 0.008*"u" + 0.008*"report" + 0.008*"let" + 0.007*"rule" + 0.007*"user" + 0.007*"moderator" + 0.006*"message" + 0.006*"bot" + 0.006*"another" + 0.006*"violation" + 0.006*"girl" + 0.006*"daddy" + 0.006*"never" + 0.005*"old"'),
 (1,
  u'0.049*"like" + 0.040*"look" + 0.019*"roast" + 0.014*"guy" + 0.013*"school" + 0.012*"bet" + 0.011*"dick" + 0.010*"as" + 0.010*"fuck" + 0.009*"friend" + 0.009*"kid" + 0.008*"girl" + 0.007*"would" + 0.007*"fat" + 0.007*"nice" + 0.006*"say" + 0.006*"want" + 0.006*"one" + 0.005*"got" + 0.005*"people"'),
 (2,
  u'0.019*"make" + 0.015*"face" + 0.009*"eyebrow" + 0.008*"day" + 0.008*"one" + 0.008*"made" + 0.007*"see" + 0.007*"know" + 0.007*"hair" + 0.006*"mouth" + 0.006*"year" + 0.006*"think" + 0.006*"like" + 0.006*"bad" + 0.006*"time" + 0.005*"feel" + 0.005*"even" + 0.005*"get" + 0.005*"would" + 0.005*"thing"'),
 (3,
  u'0.084*"look" + 0.083*"like" + 0.016*"face

In [36]:
#LDA-10
%time lda_model = ldamodel.LdaModel(corpus, num_topics=10, id2word=dictionary)
lda_model.save('tmp/cyberbullying_ldaModel.lda')
lda_model.print_topics(10,20)

CPU times: user 2.55 s, sys: 8 ms, total: 2.56 s
Wall time: 2.55 s


[(0,
  u'0.078*"look" + 0.062*"like" + 0.034*"thing" + 0.034*"guy" + 0.028*"hair" + 0.023*"abraham" + 0.023*"lincoln" + 0.023*"make" + 0.023*"picture" + 0.023*"kid" + 0.022*"really" + 0.017*"year" + 0.017*"still" + 0.017*"work" + 0.017*"last" + 0.017*"people" + 0.017*"seen" + 0.017*"chin" + 0.012*"ugly" + 0.012*"nice"'),
 (1,
  u'0.061*"like" + 0.038*"know" + 0.031*"also" + 0.027*"would" + 0.026*"post" + 0.023*"comment" + 0.019*"look" + 0.019*"face" + 0.019*"think" + 0.019*"got" + 0.019*"long" + 0.019*"looking" + 0.019*"still" + 0.019*"halloween" + 0.019*"thing" + 0.019*"girl" + 0.016*"guy" + 0.016*"go" + 0.016*"amp" + 0.016*"really"'),
 (2,
  u'0.049*"like" + 0.042*"wow" + 0.035*"look" + 0.028*"improvement" + 0.021*"though" + 0.021*"doritos" + 0.021*"oh" + 0.021*"need" + 0.015*"get" + 0.015*"time" + 0.015*"much" + 0.015*"want" + 0.015*"always" + 0.015*"think" + 0.015*"nice" + 0.015*"could" + 0.015*"looking" + 0.015*"u" + 0.015*"photo" + 0.015*"take"'),
 (3,
  u'0.057*"get" + 0.048*"lo

In [14]:
lda_features = []
for docbow in corpus:
    lda_features.append(lda_model[docbow][0][1])

#print(lda_features[:100])

<h3>Bad Words Features</h3>

In [ ]:
bad_words_ids = [word_id for word_id, word in dictionary.iteritems() if word in bad_words]
bad_words_features = []

for docbow in corpus:
        bad_words_features.append(sum([freq for word_id,freq in docbow if word_id in bad_words_ids]))
#print(bad_words_features[:100])

<h1>Validación Cruzada</h1>

<h3>Etiquetas</h3>
<p>Asignar etiqueta segun corpus</p>

In [42]:
y1 = [0 for label in range(48859)]
y2 = [1 for label in range(48859,len(corpus))]
y = y1 + y2
print(len(y))

601


In [45]:
from sklearn.cross_validation import train_test_split
X = zip(lda_features,bad_words_features)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .7)

from sklearn.neighbors import KNeighborsClassifier
myClassifier = KNeighborsClassifier()
myClassifier.fit(X_train, y_train)
predictions = myClassifier.predict(X_test)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, predictions))

0.515439429929


/home/sebastian/anaconda3/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
#def classifyPost(document):
#LDA
